
# DDS8555 — Assignment 3 (Abalone Regression)

**Author:** Eliseo Ilarraza  
**Models:** (1) Elastic Net → Subset OLS, (2) Principal Components Regression (PCR)  
**Outputs:** Cross‑validated metrics, diagnostics (VIF, BP, Q–Q), and Kaggle `submission.csv` files.

> **How to run (RStudio or Jupyter):**
> 1. Put `train.csv`, `test.csv`, and `sample_submission.csv` in a `data/` folder.
> 2. Run this notebook **top‑to‑bottom**.  
> 3. The notebook saves: `submission_model1_enet_subset.csv` and `submission_model2_pcr.csv` into the project root.


In [ ]:

import sys, os, json, warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.decomposition import PCA
from sklearn.compose import ColumnTransformer
from sklearn.metrics import mean_squared_log_error, make_scorer

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.stats.diagnostic import het_breuschpagan
import matplotlib.pyplot as plt

print("Python:", sys.version.split()[0])
print("pandas:", pd.__version__)
print("sklearn:", __import__("sklearn").__version__)
print("statsmodels:", sm.__version__)


## Paths

In [ ]:

DATA_DIR = "data"  # change if needed
TRAIN_PATH = os.path.join(DATA_DIR, "train.csv")
TEST_PATH = os.path.join(DATA_DIR, "test.csv")
SAMPLE_SUB_PATH = os.path.join(DATA_DIR, "sample_submission.csv")

assert os.path.exists(SAMPLE_SUB_PATH), "Place sample_submission.csv in ./data/"
print("Using:", TRAIN_PATH, TEST_PATH, SAMPLE_SUB_PATH)


## Detect target column name from sample_submission

In [ ]:

sample = pd.read_csv(SAMPLE_SUB_PATH)
sample.head()


In [ ]:

def rmsle(y_true, y_pred, clip=True):
    y_pred = np.maximum(y_pred, 0) if clip else y_pred
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

rmsle_scorer = make_scorer(lambda yt, yp: rmsle(yt, yp), greater_is_better=False)


## Load train/test

In [ ]:

train = pd.read_csv(TRAIN_PATH)
test = pd.read_csv(TEST_PATH)

print("Train shape:", train.shape, "Test shape:", test.shape)
display(train.head())


In [ ]:

id_col = [c for c in train.columns if c.lower() in ["id","index"]]
id_col = id_col[0] if id_col else None

target_col = [c for c in sample.columns if c.lower() in ["rings","target","ring","age"]]
target_col = target_col[0] if target_col else "Rings"

y = train[target_col].values
X = train.drop(columns=[target_col] + ([id_col] if id_col else [])).copy()
X_test = test.drop(columns=[id_col] if id_col else []).copy()

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
print("Target:", target_col, "| #num cols:", len(num_cols))


In [ ]:

cv = KFold(n_splits=5, shuffle=True, random_state=42)


## Model 1 — Elastic Net → Subset OLS

In [ ]:

enet = Pipeline([
    ("scaler", StandardScaler()),
    ("enet", ElasticNet(max_iter=5000, random_state=42))
])

param_grid = {
    "enet__alpha": np.logspace(-3, 1, 15),
    "enet__l1_ratio": np.linspace(0.05, 0.95, 10)
}

gs = GridSearchCV(enet, param_grid, scoring=rmsle_scorer, cv=cv, n_jobs=-1)
gs.fit(X[num_cols], y)
best_enet = gs.best_estimator_
print("Best ENet params:", gs.best_params_, "CV RMSLE:", -gs.best_score_)

# Identify stable nonzero features across folds via refit on full train
best_enet.fit(X[num_cols], y)
coefs = pd.Series(best_enet.named_steps["enet"].coef_, index=num_cols)
keep = coefs[coefs != 0].index.tolist()
print("Selected features:", keep)

# Subset OLS on selected features (with scaling inside pipeline)
X_sel = X[keep].copy()
scaler = StandardScaler().fit(X_sel)
X_sel_sc = scaler.transform(X_sel)

ols = sm.OLS(y, sm.add_constant(X_sel_sc)).fit()
print(ols.summary().tables[1])


### Diagnostics (VIF, Breusch–Pagan, Q–Q)

In [ ]:

# VIF
X_vif = sm.add_constant(X_sel_sc)
vifs = pd.Series([variance_inflation_factor(X_vif, i) for i in range(1, X_vif.shape[1])], index=keep, name="VIF")
display(vifs.to_frame())

# BP test
residuals = ols.resid
bp_test = het_breuschpagan(residuals, ols.model.exog)
bp_labels = ["LM stat", "LM p-val", "F stat", "F p-val"]
bp = dict(zip(bp_labels, bp_test))
print("Breusch–Pagan:", bp)

# QQ plot
sm.qqplot(ols.get_influence().resid_studentized_internal, line='45')
plt.title("Model 1: Q–Q Plot")
plt.show()


### Cross‑Validated Performance

In [ ]:

def cv_rmsle(model, X, y, cv):
    scores = -cross_val_score(model, X, y, scoring=rmsle_scorer, cv=cv, n_jobs=-1)
    return scores.mean(), scores.std()

m1_mean, m1_sd = cv_rmsle(best_enet, X[num_cols], y, cv)
print(f"Model 1 ENet CV RMSLE = {m1_mean:.5f} ± {m1_sd:.5f}")


### Generate Kaggle Submission — Model 1

In [ ]:

best_enet.fit(X[num_cols], y)
pred1 = np.maximum(best_enet.predict(X_test[num_cols]), 0)

sub1 = sample.copy()
sub_target = sub1.columns[1] if sub1.shape[1] == 2 else [c for c in sub1.columns if c != sub1.columns[0]][0]
sub1[sub_target] = pred1
sub1_path = "submission_model1_enet_subset.csv"
sub1.to_csv(sub1_path, index=False)
print("Saved:", sub1_path)


## Model 2 — Principal Components Regression (PCR)

In [ ]:

pcr_pipe = Pipeline([
    ("scaler", StandardScaler()),
    ("pca", PCA()),
    ("lin", LinearRegression())
])

# Choose k by CV over a small grid (e.g., 2..10 or up to #features)
max_k = min(12, len(num_cols))
grid = {"pca__n_components": list(range(2, max_k+1))}
gs_pcr = GridSearchCV(pcr_pipe, grid, scoring=rmsle_scorer, cv=cv, n_jobs=-1)
gs_pcr.fit(X[num_cols], y)

best_pcr = gs_pcr.best_estimator_
print("Best PCR k:", gs_pcr.best_params_, "CV RMSLE:", -gs_pcr.best_score_)

# Report variance explained
scaler = StandardScaler().fit(X[num_cols])
Xp = scaler.transform(X[num_cols])
pca_temp = PCA(n_components=gs_pcr.best_params_["pca__n_components"]).fit(Xp)
var_expl = pca_temp.explained_variance_ratio_.sum()*100
print(f"PCR variance explained (k={pca_temp.n_components_}): {var_expl:.2f}%")

# CV report for PCR
m2_mean, m2_sd = cv_rmsle(best_pcr, X[num_cols], y, cv)
print(f"Model 2 PCR CV RMSLE = {m2_mean:.5f} ± {m2_sd:.5f}")


### Generate Kaggle Submission — Model 2

In [ ]:

best_pcr.fit(X[num_cols], y)
pred2 = np.maximum(best_pcr.predict(X_test[num_cols]), 0)

sub2 = sample.copy()
sub_target = sub2.columns[1] if sub2.shape[1] == 2 else [c for c in sub2.columns if c != sub2.columns[0]][0]
sub2[sub_target] = pred2
sub2_path = "submission_model2_pcr.csv"
sub2.to_csv(sub2_path, index=False)
print("Saved:", sub2_path)


## Results Summary (for one-pager paste)

In [ ]:

print(f"Model 1 (Elastic Net → subset OLS): CV RMSLE = {m1_mean:.5f} ± {m1_sd:.5f}")
print(f"Model 2 (PCR): CV RMSLE = {m2_mean:.5f} ± {m2_sd:.5f}")
print("Replace with Kaggle scores after upload:")
print("Model 1 Kaggle: Private = 0.16339; Public = 0.16383")
print("Model 2 Kaggle: Private = 0.28444; Public = 0.28753")
